# Time-Scaling LSTM

...

## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import sys
import os.path as osp
import joblib
from dateutil.relativedelta import relativedelta
sys.path.append("../src")
from models.moisture_rnn import RNNData
import pandas as pd
from utils import str2time, read_yml
from data_funcs import cv_data_wrap
from viz import plot_styles

In [ ]:
model_dir = "../models/rocky23-24"
rnn = tf.keras.models.load_model(osp.join(model_dir, "rnn.keras"))
scaler = joblib.load(osp.join(model_dir, "scaler.joblib"))
conf = read_yml(osp.join(model_dir, "train_config.yaml"))
features_list = conf["features_list"]

dat = pd.read_pickle(osp.join(model_dir, "ml_data.pkl"))

In [ ]:
tstart = str2time(conf["train_start"])
tend = tstart + relativedelta(days = 31)

In [ ]:
rnn.summary()

## Training Data Summary

Scale it

In [ ]:
train, val, test = cv_data_wrap(dat, fstart=None, fend=None, tstart=tstart, tend=tend, val_hours=48, test_frac = 0.1, random_state=42)
rnndat = RNNData(train, val, test=None, method="random", timesteps=48, random_state=None, features_list = features_list, scaler = "standard")

In [ ]:
# Change to target scaler:
rnndat.scaler = scaler
rnndat.scale_data()

## Test Longrun Behavior

Based on features list, set up constant inputs. Since standard scaling used, start with zeros input to represent mean of all features.

In [ ]:
tsteps = 500
X0 = np.zeros((1, tsteps, len(features_list)))

In [ ]:
p0 = rnn.predict(X0)

In [ ]:
plt.plot(np.arange(0, tsteps), p0[0,:,0])
plt.xlabel("Time Step")
plt.ylabel("FMC Prediction (%)")
plt.title("Long-run behavior with constant zeros")

In [ ]:
print(f"Last 10 values: {p0[0,-10:,0]}")

In [ ]:
plt.plot(np.arange(0, 30), p0[0,0:30,0])
plt.axvline(x=3, linestyle="dashed", color="k")
plt.axvline(x=2, linestyle="dashed", color="k")
plt.xlabel("Time Step")
plt.ylabel("FMC Prediction (%)")
plt.title("Starting timesteps")

### Add spinup 


In [ ]:
spinup=3

X0 = np.zeros((1, tsteps, len(features_list)))

## Attempt direct time-scale

Modify forget gate and input gate biases.

The LSTM weights are a list of 3 arrays: input weights, recurrent weights, and biases. Within each array, based on stackoverflow thread, the order is `i, f, c, o` which stands for input gate, forget gate, cell gate and output gate respectively. This applies to the elements within a list of weights.

In [ ]:
lstm = rnn.get_layer("lstm")
lweights = lstm.get_weights()

In [ ]:
# Input Weights: 4 gates, Expect (N Features, 4 x N) Units as shape
print(f"N. Features: {len(features_list)}")
print(f"N. LSTM Units: {lstm.units}")
print(f"Input Layer Shape: {lweights[0].shape}")

In [ ]:
# Recurrent Weights: 4 gates, Expect (N Units, 4 x N Units) as shape
print(f"N. LSTM Units: {lstm.units}")
print(f"Recurrent Layer Shape: {lweights[1].shape}")

In [ ]:
# Biases: 4 gates, expect (4*N Units, ) shape
print(f"N. LSTM Units: {lstm.units}")
print(f"Recurrent Layer Shape: {lweights[2].shape}")

In [ ]:
# Modify biases for slow-down (example: -0.5 to input, +0.5 to forget)
lweights2 = lweights.copy()
b = lweights[2].copy()

# Input gate biases (i)
b[0:lstm.units] -= 0.5

# Forget gate biases (f)
b[lstm.units:2*lstm.units] += 0.5

# Update the bias in the weights list
lweights2[2] = b

# Now set these weights into the same layer in rnn2
rnn.get_layer("lstm").set_weights(lweights2)

In [ ]:
p1 = rnn.predict(X0)

In [ ]:
plt.plot(np.arange(0, 500), p0[0,0:500,0], label="orig")
plt.plot(np.arange(0, 500), p1[0,0:500,0], label="scaled")
plt.xlabel("Time Step")
plt.ylabel("FMC Prediction (%)")
plt.title("Starting timesteps")
plt.legend()

In [ ]:
lweights3 = lweights.copy()

# Modify biases for slow-down (example: -0.5 to input, +0.5 to forget)
b3 = lweights[2].copy()

# Input gate biases (i)
b3[0:lstm.units] += .5

# Forget gate biases (f)
b3[lstm.units:2*lstm.units] -= .5

# Update the bias in the weights list
lweights3[2] = b3

# Now set these weights into the same layer in rnn2
rnn.get_layer("lstm").set_weights(lweights3)

In [ ]:
p2 = rnn.predict(X0)

In [ ]:
plt.plot(np.arange(0, 50), p0[0,0:50,0], label="orig")
plt.plot(np.arange(0, 50), p1[0,0:50,0], label="scaled - slow")
plt.plot(np.arange(0, 50), p2[0,0:50,0], label="scaled - fast")
plt.xlabel("Time Step")
plt.ylabel("FMC Prediction (%)")
plt.title("Starting timesteps")
plt.legend()

## Test with Real Data

Get some stretches with no rain at first.

In [ ]:
samples = [0, 1, 2, 3]

XX = rnndat.X_train[samples,:,:]
yy = rnndat.y_train[samples,:,:]

# Get corresponding raw data for viz


In [ ]:
# Plot them to make sure it's nice
ed = features_list.index("Ed")
ew = features_list.index("Ew")
rain = features_list.index("rain")
ind=0

plt.plot(XX[samples[1], :, ed], **plot_styles["Ed"])
plt.plot(XX[samples[1], :, ew], **plot_styles["Ew"])
plt.plot(XX[samples[1], :, rain], **plot_styles["rain"])

In [ ]:
# predictions with different weights

rnn.get_layer("lstm").set_weights(lweights)
preds = rnn.predict(XX)

rnn.get_layer("lstm").set_weights(lweights2)
preds2 = rnn.predict(XX)

rnn.get_layer("lstm").set_weights(lweights3)
preds3 = rnn.predict(XX)

In [ ]:
plt.plot(preds[0], label="orig")
plt.plot(preds2[0], label="scaled - slow")
plt.plot(preds3[0], label="scaled - fast")
plt.xlabel("Time Step")
plt.ylabel("FMC Prediction (%)")
plt.title("Starting timesteps")
plt.legend()

In [ ]:
plt.plot(preds[1], label="orig")
plt.plot(preds2[1], label="scaled - slow")
plt.plot(preds3[1], label="scaled - fast")
plt.xlabel("Time Step")
plt.ylabel("FMC Prediction (%)")
plt.title("Starting timesteps")
plt.legend()

In [ ]:
plt.plot(preds[2], label="orig")
plt.plot(preds2[2], label="scaled - slow")
plt.plot(preds3[2], label="scaled - fast")
plt.xlabel("Time Step")
plt.ylabel("FMC Prediction (%)")
plt.title("Starting timesteps")
plt.legend()

In [ ]:
plt.plot(preds[3], label="orig")
plt.plot(preds2[3], label="scaled - slow")
plt.plot(preds3[3], label="scaled - fast")
plt.xlabel("Time Step")
plt.ylabel("FMC Prediction (%)")
plt.title("Starting timesteps")
plt.legend()